In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- Selected Channels: 15 (Fp1, Fp2, F3, F4, Cz, P3, P4, C3, C4, Cz, P3, P4, Pz, O1, O2)
- Range of Delta (for Artificial Data Generation): [0.0001, 0.0005]
- Input size: (65, 198, 15)
- Input type: Short Time Fourier Transform with window length 128, overlap 100
- Number of samples: 1212 (1200 fake samples + 12 real samples)

In [2]:
import numpy as np

path = '/content/drive/My Drive/ML/'
stft = np.load(path + 'Fstft3.npy')

In [3]:
num_classes = 4

output_values = np.concatenate((np.tile(0, 303), np.tile(1, 303), np.tile(2, 303), np.tile(3, 303)))
output_values = np.eye(num_classes)[output_values]

data_array = stft

In [4]:
data_array.shape

(1212, 15, 65, 198)

In [12]:
from scipy.ndimage import zoom

def resize_stft_matrix(stft_matrix, target_shape):
    num_samples, num_channels, spec_rows, spec_cols = stft_matrix.shape

    # Compute the scale factors for each dimension
    scale_factors = (target_shape[0] / spec_rows, target_shape[1] / spec_cols)

    # Perform bicubic interpolation to resize the STFT matrix
    resized_stft_matrix = zoom(stft_matrix, (1, 1, *scale_factors), mode='reflect', order=3)

    return resized_stft_matrix

# Assuming 'stft_matrix' is the input STFT matrix of size (1200, 15, 65, 120)
target_shape = (65, 65)

resized_stft_matrix = resize_stft_matrix(data_array, target_shape)


In [13]:
data_array = resized_stft_matrix
data_array.shape

(1212, 15, 65, 65)

In [14]:
import keras

In [15]:
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

In [16]:
# Parameters
train_params = {'dim': (65,65),
                'batch_size': 64,
                'n_classes':4,
                'n_channels': 15,
                'shuffle': True,
                'input_type': 'stft',
                'augment': True
               }
val_params = {'dim': (65,65),
              'batch_size': 64,
              'n_classes':4,
              'n_channels': 15,
              'shuffle': False,
              'input_type': 'stft',
              'augment': False
             }

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate

def mish(inputs):
  x = tf.nn.softplus(inputs)
  x = tf.nn.tanh(x)
  x = tf.multiply(x, inputs)
  return x


stft_input = keras.Input(shape=(65, 65, 15))
x = Conv2D(32, 3, activation=mish)(stft_input)
x = BatchNormalization()(x)
x = Conv2D(32, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(32, 5, strides=2, padding='same', activation=mish)(x)
x  = Concatenate(axis=-1)([MaxPooling2D((1, 2))(x), AveragePooling2D((1, 2))(x)])
x = Dropout(0.4)(x)

x = Conv2D(64, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, activation=mish)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 5, strides=2, padding='same', activation=mish)(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

x = Conv2D(128, 4, activation=mish)(x)
# x  = Concatenate(axis=-1)([MaxPooling2D((1, 2))(x), AveragePooling2D((1, 2))(x)])
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dropout(0.4)(x)

stft_prediction = Dense(4, activation='softmax')(x)

stft_model = Model(stft_input, stft_prediction)

stft_model.compile(optimizer = 'adam',
                    loss='categorical_crossentropy',
                    metrics=["accuracy"])

stft_model.summary()

stft_annealer = LearningRateScheduler(lambda x: 1e-3 * 0.97 ** x)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 65, 65, 15)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 63, 63, 32)   4352        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 63, 63, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [18]:
from sklearn.model_selection import train_test_split

data_array.shape

input = np.abs(np.transpose(data_array[:, :, :, :], (0, 2, 3, 1)))
print(input.shape)

X_train_temp, X_test, Y_train_temp, Y_test = train_test_split(input, output_values, test_size=0.3, random_state=42)
X_train, X_validate, Y_train, Y_validate = train_test_split(X_train_temp, Y_train_temp, test_size=0.25, random_state=36)

(1212, 65, 65, 15)


In [19]:
stft_history = stft_model.fit(X_train, Y_train, batch_size = 64, epochs = 300,
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[stft_annealer], class_weight = {})

Epoch 1/300
10/10 - 30s - loss: 2.2587 - accuracy: 0.2752 - val_loss: 1.3883 - val_accuracy: 0.2406 - lr: 0.0010 - 30s/epoch - 3s/step
Epoch 2/300
10/10 - 25s - loss: 2.2659 - accuracy: 0.2657 - val_loss: 1.3910 - val_accuracy: 0.2406 - lr: 9.7000e-04 - 25s/epoch - 3s/step
Epoch 3/300
10/10 - 26s - loss: 2.1194 - accuracy: 0.3176 - val_loss: 1.3908 - val_accuracy: 0.2406 - lr: 9.4090e-04 - 26s/epoch - 3s/step
Epoch 4/300
10/10 - 27s - loss: 1.8854 - accuracy: 0.3931 - val_loss: 1.3978 - val_accuracy: 0.2689 - lr: 9.1267e-04 - 27s/epoch - 3s/step
Epoch 5/300
10/10 - 24s - loss: 1.4825 - accuracy: 0.4843 - val_loss: 1.4506 - val_accuracy: 0.2689 - lr: 8.8529e-04 - 24s/epoch - 2s/step
Epoch 6/300
10/10 - 25s - loss: 1.1425 - accuracy: 0.5755 - val_loss: 1.5584 - val_accuracy: 0.2689 - lr: 8.5873e-04 - 25s/epoch - 3s/step
Epoch 7/300
10/10 - 26s - loss: 0.8532 - accuracy: 0.6855 - val_loss: 1.6867 - val_accuracy: 0.2689 - lr: 8.3297e-04 - 26s/epoch - 3s/step
Epoch 8/300
10/10 - 25s - loss:

In [20]:
test_loss, test_accuracy = stft_model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.9313187003135681
